In [1]:
using DifferentialEquations
using Plots

In [2]:
mutable struct Drone
    position::Array{Float64,1}
    velocity::Array{Float64,1}
    orientation::Array{Float64,1} # roll, pitch, yaw in radians
    angular_velocity::Array{Float64,1}
end

In [3]:
drone = Drone([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0])

Drone([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0])

In [4]:
function drone_dynamics!(du, u, p, t)
    gravity = 9.81 # gravitational constant, m/s^2
    mass = 1.0 # drone mass in kg
    force_per_rpm = 0.001 # constant: force generated by one propeller at 1 RPM : can be changed
    distance_to_propellers = 0.5 # distance from the drone's center to each propeller : can be changed

    rpms = p(t) # propeller speeds in RPM at time t
    ### force generated by each propeller is squared to account for the fact that the force is proportional to the square of the propeller speed
    forces = [force_per_rpm*rpms[1]^2, force_per_rpm*rpms[2]^2, force_per_rpm*rpms[3]^2, force_per_rpm*rpms[4]^2] # force generated by each propeller

    net_force = sum(forces) - mass*gravity # net force (upward force - weight)
    net_torque_pitch = (forces[1] + forces[4] - forces[2] - forces[3])*distance_to_propellers # net torque around x-axis
    net_torque_roll = (forces[1] + forces[2] - forces[3] - forces[4])*distance_to_propellers # net torque around y-axis
    net_torque_yaw = (forces[1] + forces[3] - forces[2] - forces[4])*distance_to_propellers # net torque around z-axis

    moment_of_inertia = mass * distance_to_propellers^2 # placeholder: moment of inertia around y-axis

    du[1:3] = u[4:6] # position updates with velocity

    # Update the x and z velocity and position based on pitch roll and yaw torques
    # du[4] = net_t

    # du[1] = du[1]*cos(u[6]) - du[2]*sin(u[6])
    # du[2] = du[1]*sin(u[6]) + du[2]*cos(u[6])

    du[4:6] = [u[7]*cos(u[6]) - u[8]*sin(u[6]), u[7]*sin(u[6]) + u[8]*cos(u[6]), net_force/mass] # velocity updates with acceleration

    # Check if drone is on the ground
    if du[3] < 0.0
        du[3] = 0.0
        du[6] = 0.0
    end

    # Check if drone is upside down
    if u[3] > pi/2 || u[3] < -pi/2
        du[4:6] = [0.0, 0.0, 0.0]
    end

    du[7:9] = u[10:12] # orientation updates with angular velocity
    du[10:12] = [net_torque_roll/moment_of_inertia, net_torque_pitch/moment_of_inertia, net_torque_yaw/moment_of_inertia] # angular velocity updates with angular acceleration
end

drone_dynamics! (generic function with 1 method)

In [ ]:
function rpm_changes(t)
    base_rpm = 1500.0
    # For demonstration, make some simple periodic changes

    rpm1 = base_rpm + 500.0*sin(t)
    rpm2 = base_rpm + 500.0*sin(t + pi/2)
    rpm3 = base_rpm + 500.0*sin(t + pi)
    rpm4 = base_rpm + 500.0*sin(t + 3pi/2)
    return [rpm1, rpm2, rpm3, rpm4]
end

In [ ]:
u0 = [drone.position; drone.velocity; drone.orientation; drone.angular_velocity]
tspan = (0.0, 10.0)  # Simulate for 10 seconds

In [ ]:
problem = ODEProblem(drone_dynamics!, u0, tspan, rpm_changes)
solution = solve(problem)

In [ ]:
p1 = plot(solution.t, [u[1] for u in solution.u], label="x", xlabel="time", ylabel="position")
plot!(solution.t, [u[2] for u in solution.u], label="y")
plot!(solution.t, [u[3] for u in solution.u], label="z")
p2 = plot(solution.t, [u[4] for u in solution.u], label="v_x", xlabel="time", ylabel="velocity")
plot!(solution.t, [u[5] for u in solution.u], label="v_y")
plot!(solution.t, [u[6] for u in solution.u], label="v_z")
p3 = plot(solution.t, [u[7] for u in solution.u], label="w_x", xlabel="time", ylabel="angular velocity")
plot!(solution.t, [u[8] for u in solution.u], label="w_y")
plot!(solution.t, [u[9] for u in solution.u], label="w_z")
plot(p1, p2, p3, layout=(3,1), legend=:outertopright)

In [ ]:
animation = @animate for i=1:length(solution.t)
    plot3d([u[1] for u in solution.u[1:i]], [u[2] for u in solution.u[1:i]], [u[3] for u in solution.u[1:i]], 
        xlabel="x", ylabel="y", zlabel="z", legend=false, title="Drone trajectory")
end

# Save the animation as a gif
gif(animation, "drone_trajectory.gif", fps = 1)

In [ ]:
using JuliaSimControl
using Drones
using SparseArrays, OSQP
using ModelingToolkit
using ControlSystemsBase.Polynomials
using StaticArrays
using WGLMakie, Rotations

In [ ]:



Ta = [3, 3, 2, 2]
x = [0; 0; 0; 0]
xwp = [10 10 -10 0; 10 20 -10 0; 0 10 -5 0; 0 0 0 0]
vel = 0
px, py, pz, pyaw = optimal_trajectory_gen(Ta, x, xwp, vel)

# Wrapper for polynomial trajectory
gen_optimal_trajectory(t) = SynthesizePolynomialTrajectory(t, px, py, pz, pyaw, Ta)


# Generating structs for model and control parameters
QuadModel = QuadrotorModel(
    mass = 4.34,
    I = cat(0.0820, 0.0845, 0.1377, dims = (1, 2)),
    Ts = 0.01,
    g = 9.81,
)
ControlGains =
    GeometricControlParams(kx = 4 * 4.34, kv = 5.6 * 4.34, kr = 8.81, kOmega = 2.54);
GeometricControllers =
    Controllers(geometric_force_controller, geometric_attitude_controller)
func_sys, mtk_quad = build_quad_dynamics(QuadModel)
quad_mtk = complete(mtk_quad)

# Initial conditions
x0 = [0, 0, 0]
v0 = [0, 0, 0]
R0 = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]
ω0 = [0, 0, 0]
x0_vec = build_initial_conditions(x0, v0, R0, ω0; quad_mtk = quad_mtk, func_sys = func_sys)

#Initiallising sim
Tf = 10.0
state_arr = run_quadrotor_sim(
    Tf,
    x0_vec,
    quadrotor_dynamics!,
    func_sys,
    QuadModel,
    GeometricControllers,
    ControlGains,
    gen_optimal_trajectory,
)

time_arr = state_arr.t
x_arr = state_arr[1, 1, :]
y_arr = state_arr[2, 1, :]
z_arr = state_arr[3, 1, :]

# Generating Quaternion matrix for Makie animation
R_arr = state_arr[7:15, 1, :]
quat_arr = quadrotor_RtoQuat(R_arr)

# Setting limits of animation
Scenelim = Rect(Vec3f(-0.1, -0.1, -0.1), Vec3f(15, 15, 15))
quad_simulator(
    time_arr,
    x_arr,
    y_arr,
    z_arr,
    quat_arr;
    quad_scale = 2.0,
    limit_scene = Scenelim,
)

In [ ]:
import Pkg;
Pkg.add("ReinforcementLearning")
Pkg.add("ReinforcementLearningBase")
Pkg.add("ReinforcementLearningCore")


In [ ]:
Pkg.instantiate()

In [ ]:
using ReinforcementLearningBase, ReinforcementLearningCore

In [ ]:
struct ContinuousDroneActionSpace <: AbstractSpace
    low::Array{Float64, 1}
    high::Array{Float64, 1}
end

In [ ]:
mutable struct DroneEnv <: AbstractEnv
    drone::Drone
    target_position::Array{Float64,1}
    reward::Float64
    done::Bool
end

In [ ]:
function DroneEnv()
    drone = Drone(zeros(3), zeros(3), zeros(3), zeros(3))
    target_position = [10.0, 10.0, 10.0]
    DroneEnv(drone, target_position, 0.0, false)
end

In [ ]:
function ReinforcementLearningBase.reset!(env::DroneEnv)
    env.drone = Drone(zeros(3), zeros(3), zeros(3), zeros(3))
    env.done = false
    env.reward = 0.0
end

In [ ]:
function ReinforcementLearningBase.actions(env::DroneEnv)
    # Define the action space here. This could be a specific set of RPMs, or a continuous space, depending on your design.
end

In [ ]:
function ReinforcementLearningBase.is_terminated(env::DroneEnv)
    env.done
end

In [ ]:
function ReinforcementLearningBase.observe(env::DroneEnv)
    # Return the current state of the environment.
    # This could include drone position, velocity, orientation, and angular velocity.
    # You may want to concatenate all of these into a single vector.
end

In [ ]:
# Reward function
function ReinforcementLearningBase.reward(env::DroneEnv, action)
    # Calculate the reward here.
    # A common reward function is negative distance to target.
    # You may want to add other incentives or penalties, such as for extreme orientations or velocities.
end

function get_state(drone::Drone)
    return [drone.position..., drone.velocity..., drone.orientation..., drone.angular_velocity...]
end

function apply_changes!(drone::Drone, du)
    # This function should update the drone's state based on the changes calculated by drone_dynamics!.
    # For example:
    drone.position .+= du[1:3]
    drone.velocity .+= du[4:6]
    drone.orientation .+= du[7:9]
    drone.angular_velocity .+= du[10:12]
end

function ReinforcementLearningBase.step!(env::DroneEnv, action)
    # Apply the action (change in RPMs or rotor forces), then update drone's state.
    # Use the drone_dynamics! function here.
    du = zeros(12) #assuming the size of du matches with the state size
    drone_dynamics!(du, get_state(env.drone), (t) -> action, 0.0) # assuming that drone_dynamics! is defined in a way that is compatible with this use

    # Update drone state with the changes
    apply_changes!(env.drone, du)

    # Also update the reward and done status.
    env.reward = reward(env, action)
    env.done = is_done(env)
end

In [ ]:
ContinuousDroneActionSpace() = ContinuousDroneActionSpace([1000.0, 1000.0, 1000.0, 1000.0], [20000.0, 20000.0, 20000.0, 20000.0])

In [ ]:
function ReinforcementLearningBase.sample(space::ContinuousDroneActionSpace)
    rand(4) .* (space.high .- space.low) .+ space.low
end

In [ ]:
Base.eltype(::ContinuousDroneActionSpace) = Vector{Float64}

Base.in(a, space::ContinuousDroneActionSpace) = all(a .>= space.low) && all(a .<= space.high)

Base.rand(rng::AbstractRNG, s::SampleAsync, x::ContinuousDroneActionSpace) = x.low .+ rand(rng, length(x.low)) .* (x.high .- x.low)

ReinforcementLearningBase.n_actions(::ContinuousDroneActionSpace) = 4

In [ ]:
function ReinforcementLearningBase.actions(env::DroneEnv)
    env.action_space
end